# Control de calidad

## Cargar librerias

In [1]:
import glob  # Para buscar archivos en un directorio.
import pandas as pd  # Para manipulación de datos.
import seaborn as sns  # Para visualización de datos.
import os  # Para interactuar con el sistema operativo.
import zipfile  # Para trabajar con archivos zip.
import matplotlib.pyplot as plt  # Para visualización de datos en 2D.
import numpy as np  # Para cálculos numéricos.
from datetime import datetime, date, timedelta  # Para fechas y horas.
import requests  # Para realizar solicitudes HTTP.
import urllib.request  # Para trabajar con URLs.
import io  # Para manejar entradas/salidas.

## Un archivo a la vez

### Cargar y transformar datos desde enlace

Abrir datos

In [2]:
# Obtener la fecha actual y la fecha siguiente
fecha_actual = date.today().strftime("%Y%m%d")
fecha_siguiente = (date.today() + timedelta(days=1)).strftime("%Y%m%d")

# Formar el enlace completo
enlace_base = "https://wrf1-5.imn.ac.cr/modelo/backup/TESIS/"
enlace_completo = f"{enlace_base}{fecha_actual}/{fecha_siguiente}.lluvia.csv"

# Leer los datos directamente desde el enlace y agregar un encabezado personalizado
encabezado = ["Estacion", "1", "2", "3", "4"]  # Reemplaza con tus etiquetas de columna deseadas
data = pd.read_csv(enlace_completo, sep=",", header=None, names=encabezado)

Transformar datos

In [3]:
#Se suman las 4 columnas de precipitación en cada dataframe
data['Valor'] = data.iloc[:,1] + data.iloc[:,2] + data.iloc[:,3] + data.iloc[:,4]

# Seleccionar solo las columnas "Columna1" y "Columna2" de la tabla
data_descargada = data[["Estacion", "Valor"]]

### Abrir valores extremos de la climatología

In [7]:
# Ruta del archivo Excel
archivo_excel = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Datos/Observado/Lluvia/Climatologia_Mensual_Observada_Lluvia_20230516.xlsx'

# Cargar las tablas valor_climatologia_min y valor_climatologia_max desde el archivo Excel
tabla_min = pd.read_excel(archivo_excel, sheet_name='min')
tabla_max = pd.read_excel(archivo_excel, sheet_name='max')

# Convertir los nombres de las columnas a cadenas de texto
tabla_min.columns = tabla_min.columns.astype(str)
tabla_max.columns = tabla_max.columns.astype(str)

### Ajuste de datos con la climatologia

El código itera sobre cada estación y realiza comparaciones entre los valores del modelo y los valores extremos de la climatologia mensual.

In [24]:
# Crear una nueva tabla vacía con la fecha_siguiente y las estaciones como columnas
nueva_tabla = pd.DataFrame(columns=["Fecha"] + list(tabla_min.columns[1:]))

fecha_siguiente = datetime.strptime(fecha_siguiente, "%Y%m%d")
mes_siguiente = fecha_siguiente.month

# Agregar la columna de fecha con la variable fecha_siguiente
nueva_tabla.loc[0, "Fecha"] = fecha_siguiente.strftime("%Y%m%d")

for estacion in nueva_tabla.columns[1:]:
    # Obtener el valor de tabla_min y tabla_max para la estación y el mes siguiente
    valor_min = tabla_min.loc[tabla_min["FECHA"] == mes_siguiente, estacion].values[0]
    valor_max = tabla_max.loc[tabla_max["FECHA"] == mes_siguiente, estacion].values[0]

    # Verificar si existen filas que coincidan con la estación en datos_descargados
    filas_estacion = data_descargada.loc[data_descargada["Estacion"].astype(str).str.strip() == estacion.strip()]
    
    if not filas_estacion.empty:
        # Obtener el valor de datos_descargados para la estación y el mes siguiente
        valor_descargado = filas_estacion["Valor"].iloc[0]

        # Comparar el valor_descargado con los valores de tabla_min y tabla_max
        if valor_descargado >= valor_max:
            nueva_tabla.loc[0, estacion] = valor_max
            data_descargada.loc[filas_estacion.index, "Valor"] = valor_max
        elif valor_descargado < 0:
            nueva_tabla.loc[0, estacion] = valor_min
            data_descargada.loc[filas_estacion.index, "Valor"] = valor_min
        else:
            nueva_tabla.loc[0, estacion] = "No"
    else:
        # Si no hay filas para la estación en datos_descargados, asignar "x"
        nueva_tabla.loc[0, estacion] = "x"

# Imprimir la nueva tabla
print(nueva_tabla)

      Fecha 100633 100641 100643 100649 100651 100653 100655 200002 200007  \
0  20230615      x     No      x     No     No     No     No      x      x   

   ... 94019 96003 98075 98087 98091 98095 98097 98105 98107 98109  
0  ...     x    No    No    No  51.8     x    No     x    No    No  

[1 rows x 155 columns]


Mostrar nueva tabla

In [27]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
nueva_tabla

,Fecha,100633,100641,100643,100649,100651,100653,100655,200002,200007,200009,69633,69647,69677,69679,69681,69699,69701,69709,69711,69713,69715,69717,69719,69721,69723,69725,69727,69729,69731,69735,69737,69739,69743,69747,69749,71015,71023,72149,72153,72157,72159,72163,72165,72167,72177,72179,72181,72183,72185,72187,72189,72191,72193,72195,73123,73129,73137,73141,73143,73145,73147,73149,73151,73153,73155,73157,73159,73161,73163,73167,73169,73171,74051,74053,74061,74063,74067,74069,74071,74073,74075,74077,74081,74083,76055,76059,76061,76063,76067,78027,78031,78033,78035,80013,81005,82011,82013,82015,82017,82019,83007,84139,84141,84181,84185,84187,84189,84191,84193,84197,84199,84203,84207,84209,84213,84215,84217,84219,84225,84231,84239,84241,84243,84247,84249,84251,84283,84285,84291,84295,85021,85023,85025,85027,86013,87013,88035,88047,88049,88051,88055,90015,94013,94015,94019,96003,98075,98087,98091,98095,98097,98105,98107,98109
0,20230615,x,No,x,No,No,No,No,x,x,x,No,No,x,No,x,No,No,No,No,No,No,No,x,No,No,x,x,No,No,No,No,x,No,x,No,No,x,x,x,x,No,No,x,No,x,x,No,No,x,x,No,No,x,No,No,x,No,No,x,No,No,No,No,No,No,x,No,x,x,No,No,No,No,No,x,No,x,x,No,x,x,x,No,x,x,No,No,No,x,x,x,No,No,x,No,No,No,No,x,No,x,No,No,No,No,No,No,x,x,No,No,x,No,No,No,No,No,No,x,x,x,x,No,x,No,No,x,No,x,No,x,x,No,x,No,No,x,No,No,No,No,No,72.8,No,x,No,No,No,51.8,x,No,x,No,No


## Todos los archivos

### Cargar y transformar datos desde enlace

Abrir todos los archivos desde el enlace

In [ ]:
# Definir el enlace base para los archivos
enlace_base = "https://wrf1-5.imn.ac.cr/modelo/backup/TESIS/"


fecha_inicio = datetime(2023, 5, 9).date()  # Fecha de inicio
fecha_fin = datetime.now().date() + timedelta(days=1)  # Fecha de fin

# Inicializar las listas vacías para almacenar los DataFrames y los nombres de archivo
dataframes, file_name = [], []

# Iterar sobre el rango de fechas
fecha_actual = fecha_inicio

while fecha_actual <= fecha_fin:
    fecha_siguiente = fecha_actual + timedelta(days=1) # Obtener la fecha siguiente

    # Formar el enlace completo para el archivo CSV de la fecha actual y siguiente
    enlace_siguiente = f"{enlace_base}{fecha_actual.strftime('%Y%m%d')}/{fecha_siguiente.strftime('%Y%m%d')}.lluvia.csv"

    response = requests.get(enlace_siguiente) # Realizar la solicitud HTTP GET para obtener el archivo CSV

    if response.status_code == 200: # Verificar si la solicitud fue exitosa (código de estado 200)
        csv_data = response.content.decode('utf-8') # Obtener los datos CSV de la respuesta y decodificarlos como texto UTF-8

        # Crear un DataFrame de Pandas a partir de los datos CSV y especificar el encabezado de columnas
        df = pd.read_csv(io.StringIO(csv_data), header=None, names=["Estacion", "1", "2", "3", "4"])

        dataframes.append(df) # Agregar el DataFrame a la lista de DataFrames

        file_name.append(fecha_siguiente.strftime('%Y%m%d')) # Agregar el nombre de archivo correspondiente

    fecha_actual = fecha_siguiente # Actualizar la fecha actual para la próxima iteración del ciclo

Transformar los datos para obtener la estructura adecuada

In [155]:
for df in dataframes:
    # Sumar las 4 columnas de precipitación
    df['Valor'] = df.iloc[:, 1] + df.iloc[:, 2] + df.iloc[:, 3] + df.iloc[:, 4]

In [156]:
nuevas_dataframes = [tabla[['Estacion', 'Valor']] for tabla in dataframes]

### Cargar los valores extremos de las climatologias mensuales 

In [157]:
# Ruta del archivo Excel
archivo_excel = 'C:/Users/arias/OneDrive/Documentos/UCR/TFG/Datos/Observado/Lluvia/Climatologia_Mensual_Observada_Lluvia_20230516.xlsx'

# Cargar las tablas valor_climatologia_min y valor_climatologia_max desde el archivo Excel
tabla_min = pd.read_excel(archivo_excel, sheet_name='min')
tabla_max = pd.read_excel(archivo_excel, sheet_name='max')

# Convertir los nombres de las columnas a cadenas de texto
tabla_min.columns = tabla_min.columns.astype(str)
tabla_max.columns = tabla_max.columns.astype(str)

### Ajuste de datos con la climatologia

El código itera sobre cada estación y realiza comparaciones entre los valores del modelo y los valores extremos de la climatologia mensual.

In [ ]:
nueva_tabla = pd.DataFrame(columns=tabla_min.columns)  # Crear una nueva tabla vacía con las columnas necesarias

for tabla, fecha in zip(nuevas_dataframes, file_name):
    fecha_dt = datetime.strptime(fecha, "%Y%m%d")  # Convertir fecha a formato de fecha
    mes = fecha_dt.month

    nueva_fila = pd.DataFrame({"FECHA": [fecha_dt.strftime("%Y%m%d")]})  # Crear una nueva fila con la fecha correspondiente

    for estacion in nueva_tabla.columns[1:]:
        estacion = str(estacion)

        # Obtener el valor de tabla_min y tabla_max para la estación y el mes siguiente
        valor_min = tabla_min.loc[tabla_min["FECHA"] == mes, estacion].values[0]
        valor_max = tabla_max.loc[tabla_max["FECHA"] == mes, estacion].values[0]

        filas_estacion = tabla.loc[tabla["Estacion"].astype(str).str.strip() == estacion]  # Filtrar filas por estación

        if not filas_estacion.empty:
            valor_tabla = filas_estacion["Valor"].iloc[0]  # Obtener el valor de tabla para la estación y el mes siguiente

            if valor_tabla >= valor_max:
                nueva_fila[estacion] = valor_max
                tabla.loc[filas_estacion.index, "Valor"] = valor_max
            elif valor_tabla < 0:
                nueva_fila[estacion] = valor_min
                tabla.loc[filas_estacion.index, "Valor"] = valor_min
            else:
                nueva_fila[estacion] = "No"
        else:
            nueva_fila[estacion] = "-"  # Si no hay filas para la estación en tabla, asignar "-"

    nueva_tabla = pd.concat([nueva_tabla, nueva_fila], ignore_index=True)  # Agregar la nueva fila a la tabla

    # Generar el nombre de archivo para la tabla ajustada y guardar en nueva carpeta
    tabla.to_csv(f"C:/Users/arias/OneDrive/Documentos/UCR/TFG/Datos/Control_Calidad_Clim/{fecha}.lluvia.csv", index=False) 

In [211]:
pd.set_option('display.max_rows', None)
nueva_tabla

,FECHA,100633,100641,100643,100649,100651,100653,100655,200002,200007,...,94019,96003,98075,98087,98091,98095,98097,98105,98107,98109
0,20230510,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
1,20230511,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
2,20230512,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
3,20230513,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
4,20230514,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
5,20230515,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
6,20230516,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
7,20230517,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
8,20230518,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
9,20230519,-,No,-,No,No,No,No,-,-,...,-,No,No,No,No,-,No,-,No,No
